In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter("ignore")

In [ ]:
df=pd.read_csv("HappinessScore.csv")
df.head()

# EDA

In [ ]:
df.columns

In [ ]:
df.describe()

key observations here

mean and median are almost near in all columns There is small differnece in 75% and max values. mean few outliers are there

In [ ]:
df.dtypes

In [ ]:
df.Country.unique()

In [ ]:
df.shape

In [ ]:
df.isnull()

In [ ]:
df.isnull().sum()

In [ ]:
sns.heatmap(df.isnull())

There are no null values in the dataset

In [ ]:
dfcor=df.corr()
dfcor


In [ ]:
plt.Figure(figsize=(6,4))
sns.heatmap(dfcor,annot=True)

#corr() shows the happiness score is highly correlated and happiness rank and generousity are negatively coorelated . we may drop them if reqd.

In [ ]:
#univariate analysis

In [ ]:
sns.distplot(df["Happiness Score"])
plt.show()

In [ ]:
sns.distplot(df["Happiness Rank"])
plt.show()

In [ ]:
sns.distplot(df["Dystopia Residual"])
plt.show()

In [ ]:
#bivariate analysis

In [ ]:
plt.scatter(df["Happiness Score"],df["Dystopia Residual"])
plt.xlabel("Dystopia Residual")
plt.ylabel("Happiness Score")
plt.show()

In [ ]:
plt.scatter(df["Happiness Rank"],df["Dystopia Residual"])
plt.xlabel("Dystopia Residual")
plt.ylabel("Happiness Rank")
plt.show()

In [ ]:
plt.scatter(df["Happiness Rank"],df["Dystopia Residual"])
plt.xlabel("Dystopia Residual")
plt.ylabel("Standard Error")
plt.show()

Standard error, happiness rank are few columns that have no positive relation with the target variable

In [ ]:
sns.pairplot(df)

In [ ]:
plt.scatter(df["Generosity"],df["Dystopia Residual"])
plt.xlabel("Dystopia Residual")
plt.ylabel("Generosity")
plt.show()

In [ ]:
df.drop(["Generosity","Happiness Rank"],axis=1,inplace=True)
df

#transforming the categorical variables to numerical for data modelling using LabelEncoder

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
le=LabelEncoder()

In [ ]:
list1=['Country','Region']
for val in list1:
    df[val]=le.fit_transform(df[val].astype(str))

In [ ]:
df

In [ ]:
df.shape

# Handling outliers

In [ ]:
df["Happiness Score"].plot.box()

In [ ]:
df["Family"].plot.box()

In [ ]:
df["Standard Error"].plot.box()

In [ ]:
df["Freedom"].plot.box()

#some outliers are present

In [ ]:
from scipy.stats import zscore

In [ ]:
z=np.abs(zscore(df))
z

In [ ]:
threshold=3
print(np.where(z>3))

In [ ]:
df_new=df[(z<3).all(axis=1)]

In [ ]:
df_new

In [ ]:
df_new.skew()

little skewness is there standard error and trust so we will remove it

In [ ]:
x=df_new.iloc[:,0:-1]
x.shape

In [ ]:
from sklearn.preprocessing import power_transform

In [ ]:
x=power_transform(x,method='yeo-johnson')

In [ ]:
y=df_new.iloc[:,-1]

In [ ]:
y.shape

#scaling the datset

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
#scaling the dataset
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
scaledX=sc.fit_transform(x)

In [ ]:
# finding the best random state

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import r2_score

In [ ]:
best_rstate=0
accuracy=0
for i in range(30,200):
    x_train,x_test,y_train,y_test=train_test_split(scaledX,y,test_size=.25,random_state=i)
    mod=LinearRegression()
    mod.fit(x_train,y_train)
    predlr=mod.predict(x_test)
    tempaccu=r2_score(y_test,predlr)
    if(tempaccu>accuracy):
        accuracy=tempaccu
        best_rstate=i
        
print("Best Accuracy",accuracy*100, "Random state",best_rstate)        

In [ ]:
# using the best random state 91 

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(scaledX,y,test_size=.25,random_state=91)

# Finding the best model

Linear Regression

In [ ]:
lr=LinearRegression()
lr.fit(x_train,y_train)
predlr=lr.predict(x_test)
r2score=r2_score(y_test,predlr)
cvscore=cross_val_score(LinearRegression(),x_train,y_train,cv=5).mean()
print("Accuracy Score" ,{r2score*100} , "Cross Val Score", {cvscore*100})

SGDRegressor

In [ ]:
from sklearn.linear_model import SGDRegressor
sgd=SGDRegressor()
sgd.fit(x_train,y_train)
predsgd=sgd.predict(x_test)
r2score=r2_score(y_test,predsgd)
cvscore=cross_val_score(SGDRegressor(),x_train,y_train,cv=5).mean()
print("Accuracy Score" ,{r2score*100} , "Cross Val Score", {cvscore*100})

SVR

In [ ]:
from sklearn.svm import SVR

In [ ]:
def svmkernel(ker):
    svr=SVR(kernel=ker)
    svr.fit(x_train,y_train)
    predsvr=svr.predict(x_test)
    r2score=r2_score(y_test,predsvr)
    cvscore=cross_val_score(SVR(),x_train,y_train,cv=5).mean()
    print("Accuracy Score" ,{r2score*100} , "Cross Val Score", {cvscore*100})

In [ ]:
svmkernel('rbf')

In [ ]:
svmkernel('linear')

# conclusion

LinearRegression is best performing with accuracy 96.7%


In [ ]:
a=np.array(y_test)
a

In [ ]:
df_con=pd.DataFrame({'original':a,"predicted":predlr},index=range(len(a)))
df_con

# saving the model- Serialization

In [ ]:
# saving the prediction model

import pickle
filename="HappinessScore.pkl"
pickle.dump(predlr,open(filename,'wb'))